In [1]:
import json
import time
# import matplotlib.pyplot as plt

In [19]:
with open("test/SportsDataFinal.jsonl", "r") as read_file:
    sample_doc = json.load(read_file)
     pass

In [31]:
import logging, sys
logging.disable(sys.maxsize)

import lucene
import os
from org.apache.lucene.store import MMapDirectory, SimpleFSDirectory, NIOFSDirectory
from java.nio.file import Paths
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.document import Document, Field, FieldType
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.index import FieldInfo, IndexWriter, IndexWriterConfig, IndexOptions, DirectoryReader
from org.apache.lucene.search import IndexSearcher, BoostQuery, Query
from org.apache.lucene.search.similarities import BM25Similarity

def create_index(dir):
    start_time = time.time()
    if not os.path.exists(dir):
        os.mkdir(dir)
    store = SimpleFSDirectory(Paths.get(dir))
    analyzer = EnglishAnalyzer()
    # for stop_word in analyzer.getDefaultStopSet():
        # print(str(stop_word))
    config = IndexWriterConfig (analyzer)
    config.setOpenMode(IndexWriterConfig.OpenMode.CREATE)
    writer = IndexWriter(store, config)
    

    titleType = FieldType()
    titleType.setStored(True)
    titleType.setTokenized(True)
    titleType.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS)

    contextType = FieldType()
    contextType.setStored(False)
    contextType.setTokenized(True)
    contextType.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS)
    
    urlType = FieldType()
    urlType.setStored(True)
    urlType.setTokenized(False)
    urlType.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS)
    
    i=0
    time_taken=[]
    # print(len(sample_doc))
    for sample in sample_doc:
        i+=1
        title = sample['title']
        context = sample['text']
        url = sample['url']
        if i%20000 == 0:
            end_time = time.time()
            time_so_far = (end_time - start_time)
            time_taken.append(time_so_far)
            print('time taken to index ' + str(i) + ' Documents: ' + str(round(time_so_far, 2)) + ' Seconds')
        doc = Document()
        doc.add(Field('Title', str(title), titleType))
        doc.add(Field('Context', str(context), contextType))
        doc.add(Field('Url', str(url), urlType))
        writer.addDocument(doc)
    writer.close()
    end_time = time.time()
    total_time = end_time - start_time
    # x = []
    # for i in range(1,len(time_taken)+1):
    #     x.append(i*5000)
    # plt.plot(x,time_taken)
    # plt.ylabel('Time in seconds')
    # plt.xlabel('No of Records indexed')
    # plt.title('Lucene Indexing Time Plot')
    # plt.show()
    # plt.plot(time_taken)
    # print(time_taken)
    print('Total indexing time:' + str(round(time_so_far, 2)) + 'seconds')

def retrieve(storedir, query):
    searchDir = NIOFSDirectory(Paths.get(storedir))
    searcher = IndexSearcher(DirectoryReader.open(searchDir))
    
    parser = QueryParser('Context', EnglishAnalyzer())
    # parser.addMultiField('Title',EnglishAnalyzer())
    parsed_query = parser.parse(query)

    topDocs = searcher.search(parsed_query, 10).scoreDocs
    topkdocs = []
    for hit in topDocs:
        doc = searcher.doc(hit.doc)
        topkdocs.append({
            "Score": hit.score,
            "Title":doc.get("Title"),
            # "Context": doc.get("Context"),
            "Url": doc.get("Url")
        })
    
    print(topkdocs)


# lucene.initVM(vmargs=['-Djava.awt.headless=true'])
# create_index('SportsDataIndex/')
retrieve('SportsDataIndex/', 'Atlanta Braves manager Brian Snitker')

[{'Score': 5.445286750793457, 'Title': 'Survey: 36% more Americans expected to bet on NFL this season compared to last', 'Url': 'https://www.espn.com/chalk/story/_/id/32165147/survey-36-more-americans-expected-bet-nfl-season-compared-last'}, {'Score': 5.445286750793457, 'Title': 'Survey: 36% more Americans expected to bet on NFL this season compared to last', 'Url': 'https://www.espn.com/chalk/story/_/id/32165147/survey-36-more-americans-expected-bet-nfl-season-compared-last'}, {'Score': 5.445286750793457, 'Title': 'Survey: 36% more Americans expected to bet on NFL this season compared to last', 'Url': 'https://www.espn.com/chalk/story/_/id/32165147/survey-36-more-americans-expected-bet-nfl-season-compared-last'}, {'Score': 5.347653388977051, 'Title': 'NFL to allow betting lounges in stadiums, but no gambling', 'Url': 'http://www.espn.com/nfl/story/_/id/28751624/nfl-allow-betting-lounges-stadiums-no-gambling'}, {'Score': 5.347653388977051, 'Title': 'NFL to allow betting lounges in stad